<a href="https://colab.research.google.com/github/simulate111/Deep-Learning-in-Human-Language-Technology/blob/main/course_project_Reza_CoP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep learning in Human Language Technology Project (Template)

- Student(s) Name(s): Mohammadreza Akhtari
- Date: October 2024
- Chosen Corpus:
- Contributions (if group project): -

### Corpus information

- Description of the chosen corpus:
- Paper(s) and other published materials related to the corpus:
- Random baseline performance and expected performance for recent machine learned models:

---

## 1. Setup

In [1]:
# Your code to install and import libraries etc. here
!pip install transformers datasets evaluate
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate
import numpy as np


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00


---

## 2. Data download, sampling and preprocessing

### 2.1. Download the corpus

In [2]:
# Your code to download the corpus here
dataset = load_dataset("mteb/amazon_reviews_multi", trust_remote_code=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


amazon_reviews_multi.py:   0%|          | 0.00/6.17k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/47.0 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/61.4M [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/53.1M [00:00<?, ?B/s]

...

ja/test.jsonl:   0%|          | 0.00/3.49M [00:00<?, ?B/s]

zh/test.jsonl:   0%|          | 0.00/2.01M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

### 2.2. Sampling and preprocessing

In [3]:
# Your code for any necessary sampling and preprocessing here
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True)

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
encoded_dataset = dataset.map(preprocess_function, batched=True)

# Downsample if necessary
small_train_dataset = encoded_dataset["train"].shuffle(seed=42).select(range(10000))
small_eval_dataset = encoded_dataset["validation"].shuffle(seed=42).select(range(1000))


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/1200000 [00:00<?, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

---

## 3. Machine learning model

### 3.1. Model training

In [4]:
# Your code to train the transformer based model on the training set and evaluate the performance on the validation set here
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=5)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Updated to use eval_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.297100,1.160896,0.485000
2,1.035800,1.135612,0.512000
3,0.885000,1.200729,0.515000


TrainOutput(global_step=1875, training_loss=1.017976416015625, metrics={'train_runtime': 1084.1993, 'train_samples_per_second': 27.67, 'train_steps_per_second': 1.729, 'total_flos': 2844907358298336.0, 'train_loss': 1.017976416015625, 'epoch': 3.0})

### 3.2 Hyperparameter optimization

In [ ]:
# Define a set of hyperparameters to search over
learning_rates = [1e-5, 2e-5, 3e-5]
weight_decays = [0.0, 0.01, 0.1]

best_accuracy = 0
best_hyperparams = {}

for lr in learning_rates:
    for wd in weight_decays:
        training_args = TrainingArguments(
            output_dir="./results",
            eval_strategy="epoch",
            learning_rate=lr,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            num_train_epochs=3,
            weight_decay=wd,
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=small_train_dataset,
            eval_dataset=small_eval_dataset,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics,
        )

        trainer.train()
        eval_results = trainer.evaluate()
        accuracy = eval_results["eval_accuracy"]

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_hyperparams = {"learning_rate": lr, "weight_decay": wd}

print(f"Best hyperparameters: {best_hyperparams}")
print(f"Best accuracy: {best_accuracy}")


Epoch,Training Loss,Validation Loss,Accuracy
1,0.664600,1.429778,0.500000
2,0.475200,1.586031,0.504000
3,0.383600,1.666864,0.505000


Epoch,Training Loss,Validation Loss,Accuracy
1,0.293400,2.148019,0.469000
2,0.200700,2.456654,0.483000
3,0.166500,2.567086,0.481000


Epoch,Training Loss,Validation Loss,Accuracy
1,0.186500,3.107278,0.474000
2,0.119600,3.241512,0.488000
3,0.097400,3.292673,0.484000


Epoch,Training Loss,Validation Loss,Accuracy
1,0.214700,3.611443,0.451000
2,0.167800,3.402284,0.481000
3,0.135000,3.545088,0.478000


Epoch,Training Loss,Validation Loss,Accuracy
1,0.249500,3.215266,0.464000
2,0.174700,3.452796,0.493000
3,0.108100,3.589129,0.503000


Epoch,Training Loss,Validation Loss,Accuracy
1,0.180200,3.811271,0.474000
2,0.142200,3.951358,0.490000
3,0.087000,3.984578,0.499000


Epoch,Training Loss,Validation Loss,Accuracy
1,0.228400,3.332064,0.491000
2,0.181900,3.605623,0.489000
3,0.130900,3.619048,0.499000


Epoch,Training Loss,Validation Loss,Accuracy
1,0.206400,3.732867,0.485000


### 3.3. Evaluation on test set

In [ ]:
# Your code to evaluate the final model on the test set here
test_results = trainer.evaluate(encoded_dataset["test"])
print(test_results)

### 3.4. Cross-lingual experiments

In [ ]:
# Your code to train and evaluate the cross-lingual model
cross_lingual_results = {}
for lang in ["de", "es", "fr", "ja", "zh"]:
    test_dataset = load_dataset("mteb/amazon_reviews_multi", lang, split="test")
    encoded_test_dataset = test_dataset.map(preprocess_function, batched=True)
    results = trainer.evaluate(encoded_test_dataset)
    cross_lingual_results[lang] = results
print(cross_lingual_results)

---

## 4. Results and summary

### 4.1 Corpus insights

(Briefly discuss what you learned about the corpus and its annotation)

### 4.2 Results

(Briefly summarize your results)

### 4.3 Relation to random baseline / expected performance / state of the art

(Compare your results with the random and state-of-the-art performance)

---

## 5. Bonus Task (optional)

### 5.1. and 5.2. Model and Data selection

(Briefly describe which model was used and why. Also, describe how the test data was downsampled, include relevant code.)

### 5.3. Prompt design

(Include your final prompt here. Also, explain here all prompt engineering insights you learned during the project.)

### 5.4. Generate

In [ ]:
# Your code to run the generative model and extract predictions from the model output.

### 5.5. Evaluation and results

(Briefly summarize your results)

### 5.6 Error analysis (group projects only)

(Present the error analysis results here)